In [1]:
import imageio
from PIL import Image, ImageSequence
import os

INPUT_GIF = "my_video.gif"
OUTPUT_WEBP = "sticker.webp"

MAX_SIZE = 512
MAX_FILE_SIZE = 1024 * 1024  # 1 MB

def resize_frame(frame):
    frame = frame.convert("RGBA")
    frame.thumbnail((MAX_SIZE, MAX_SIZE), Image.LANCZOS)
    return frame

def gif_to_whatsapp_sticker(gif_path, output_path):
    gif = Image.open(gif_path)

    frames = []
    durations = []

    for frame in ImageSequence.Iterator(gif):
        frames.append(resize_frame(frame))
        durations.append(frame.info.get("duration", 40))

    frames[0].save(
        output_path,
        format="WEBP",
        save_all=True,
        append_images=frames[1:],
        duration=durations,
        loop=0,
        lossless=False,
        quality=80,
        method=6,
        minimize_size=True
    )

    size = os.path.getsize(output_path)
    print(f"Sticker size: {size / 1024:.2f} KB")

    if size > MAX_FILE_SIZE:
        print("⚠️ File too large for WhatsApp. Reduce frames or quality.")

if __name__ == "__main__":
    gif_to_whatsapp_sticker(INPUT_GIF, OUTPUT_WEBP)
    print("✅ WhatsApp sticker created:", OUTPUT_WEBP)


Sticker size: 515.28 KB
✅ WhatsApp sticker created: sticker.webp


video to sticker

In [13]:
import subprocess
import os

INPUT_VIDEO = r"abhinav_compressed.mp4"
OUTPUT_STICKER = "abhinav_sticker.webp"

MAX_DURATION = 17   # reduce duration (VERY IMPORTANT)

def video_to_whatsapp_sticker(input_video, output_webp):
    command = [
        "ffmpeg",
        "-y",
        "-i", input_video,
        "-t", str(MAX_DURATION),

        "-vf",
        (
            "scale=512:512:force_original_aspect_ratio=decrease,"
            "pad=512:512:(ow-iw)/2:(oh-ih)/2:color=0x00000000,"
            "fps=10"   # lower FPS = smaller size
        ),

        "-loop", "0",
        "-an",
        "-vsync", "0",

        "-c:v", "libwebp",
        "-lossless", "0",
        "-q:v", "60",              # lower quality
        "-compression_level", "6",
        "-preset", "picture",      # better compression

        output_webp
    ]

    subprocess.run(command, check=True)

    size_kb = os.path.getsize(output_webp) / 1024
    print(f"✅ Sticker created: {output_webp}")
    print(f"📦 Size: {size_kb:.2f} KB")

    if size_kb > 1000:
        print("⚠️ Sticker exceeds WhatsApp 1MB limit!")

if __name__ == "__main__":
    video_to_whatsapp_sticker(INPUT_VIDEO, OUTPUT_STICKER)


✅ Sticker created: abhinav_sticker.webp
📦 Size: 978.89 KB


change speed of the video

In [7]:
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import speedx

input_video = "abhinav_kili.mp4"
output_path = "abhinav_speed.mp4"

clip = VideoFileClip(input_video)

final = clip.fx(speedx, 4)  # 2x speed

final.write_videofile(output_path)


Moviepy - Building video abhinav_speed.mp4.
MoviePy - Writing audio in abhinav_speedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video abhinav_speed.mp4



Moviepy - Done !
Moviepy - video ready abhinav_speed.mp4


change the resolution of the image with size

In [8]:
from moviepy.editor import VideoFileClip

input_video = "abhinav_speed.mp4"
output_video = "abhinav_compressed.mp4"

clip = VideoFileClip(input_video)

# Reduce resolution
resized = clip.resize(height=240)
resized.write_videofile(
    output_video,
    codec="libx264",
    audio_codec="aac",
    ffmpeg_params=["-crf", "32"],   # 18=high quality, 28=small size
    preset="slow"
)



Moviepy - Building video abhinav_compressed.mp4.
MoviePy - Writing audio in abhinav_compressedTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video abhinav_compressed.mp4



Moviepy - Done !
Moviepy - video ready abhinav_compressed.mp4


crop video by time

In [ ]:
from moviepy.editor import VideoFileClip

input_video = "input.mp4"
output_video = "trimmed.mp4"

clip = VideoFileClip(input_video)

# start_time, end_time in seconds
start_time = 10
end_time = 30

trimmed = clip.subclip(start_time, end_time)

trimmed.write_videofile(
    output_video,
    codec="libx264",
    audio_codec="aac"
)
